In [1]:
!pip install pandas torch transformers numpy


# Test model without reference (XLM-roberta)

In [35]:
#This code works
# I think it does a decent job in classifiying!

import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import XLMRobertaForSequenceClassification


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")




# Load the pre-trained model architecture
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-large",  num_labels=9)  
# Load your custom saved weights
model.load_state_dict(torch.load("/work/IdaHeleneDencker#2808/Redoing_counterspeech_classifier/output/model_without_reference/checkpoints/model_epoch_3.pt"))
# Ensure the model is in evaluation mode
model.eval()




# Simple list of texts to classify
data = ['Test', 
        'I hate you', 
        'I love you',
        '@Acosta You are more than right Jim.  Such a shame that there are so many ignorant fools in this country.', 
        'i very much agree, i think that is a great point', 
        'i think you should watch your tone, it can be hurtful to other peple',
        'Making fun of a serious health condition is not the way to get your point across.']
# Convert to DataFrame
df = pd.DataFrame(data, columns=['text'])




def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Make sure to disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(**inputs)  # Forward pass through the model

    # Get predicted class
    logits = outputs.logits

    # Split the logits for each task
    logits_hateful = logits[:, :3]          
    logits_constructive = logits[:, 3:6] 
    logits_agree = logits[:, 6:]
    #print("logits agree",logits_agree)
    
    preds_hateful = torch.argmax(logits_hateful, dim=1).item()
    preds_constructive = torch.argmax(logits_constructive, dim=1).item()
    preds_agree = torch.argmax(logits_agree, dim=1).item()
    #print("preds agree",preds_agree)
    
    return preds_hateful, preds_constructive, preds_agree



# List to store the predicted classes
predicted_class_hateful = []
predicted_class_constructive = []
predicted_class_agree = []



# Iterate over each text for prediction
for text in tqdm(df['text'], desc="Applying Model"):
    result = predict(text)
    #print(result)
    # Append to the list
    predicted_class_hateful.append(result[0])
    predicted_class_constructive.append(result[1])
    predicted_class_agree.append(result[2])


# Add the predicted classes as a new column in the DataFrame
df['predicted_class_hateful'] = predicted_class_hateful
df['predicted_class_constructive'] = predicted_class_constructive
df['predicted_class_agree'] = predicted_class_agree


label_mappings = {
    'predicted_class_hateful': {0: 'Hateful', 1: 'Not Hateful', 2:'Unclassifiable Hateful'},
    'predicted_class_constructive': {0: 'Not Constructive', 1: 'Constructive',2: 'Unclassifiable Constructive'},
    'predicted_class_agree': {0: 'Agree', 1: 'Disagree',2:  'Unclear'}}
for column, mapping in label_mappings.items():
    df[column] = df[column].map(mapping)


# Display the DataFrame with predictions
df


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_622/3182014777.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicit

,text,predicted_class_hateful,predicted_class_constructive,predicted_class_agree
0,Test,Unclassifiable Hateful,Not Constructive,Unclear
1,I hate you,Hateful,Not Constructive,Disagree
2,I love you,Not Hateful,Constructive,Agree
3,@Acosta You are more than right Jim. Such a s...,Not Hateful,Not Constructive,Agree
4,"i very much agree, i think that is a great point",Not Hateful,Not Constructive,Agree
5,"i think you should watch your tone, it can be ...",Not Hateful,Constructive,Disagree
6,Making fun of a serious health condition is no...,Not Hateful,Constructive,Disagree


# Test model with reference (XLM-roberta)

In [11]:
# use if model apllied is using the default roberta model


import torch.nn as nn
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, XLMRobertaModel
from transformers import XLMRobertaForSequenceClassification


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")



# Load the pre-trained model architecture
model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-large",  num_labels=9) 


# Load your custom saved weights
model.load_state_dict(torch.load("/work/IdaHeleneDencker#2808/Redoing_counterspeech_classifier/output/model_with_reference/checkpoints/model_epoch_4.pt"))
# Ensure the model is in evaluation mode
model.eval()



# Simple list of texts to classify
data = ['Test', 
        'I hate you', 
        'I love you',
        '@Acosta You are more than right Jim.  Such a shame that there are so many ignorant fools in this country.', 
        'i very much agree, i think that is a great point', 
        'i think you should watch your tone, it can be hurtful to other peple',
        'Making fun of a serious health condition is not the way to get your point across.']
# Convert to DataFrame
df = pd.DataFrame(data, columns=['text'])




def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    # Make sure to disable gradient calculation for inference
    with torch.no_grad():
        outputs = model(**inputs)  # Forward pass through the model 
        
    # Get predicted class
    logits = outputs.logits

    # Split the logits for each task
    logits_hateful = logits[:, :3]          
    logits_constructive = logits[:, 3:6] 
    logits_agree = logits[:, 6:]
    #print("logits agree",logits_agree)
    
    preds_hateful = torch.argmax(logits_hateful, dim=1).item()
    preds_constructive = torch.argmax(logits_constructive, dim=1).item()
    preds_agree = torch.argmax(logits_agree, dim=1).item()
    #print("preds agree",preds_agree)
    
    return preds_hateful, preds_constructive, preds_agree



# List to store the predicted classes
predicted_class_hateful = []
predicted_class_constructive = []
predicted_class_agree = []



# Iterate over each text for prediction
for text in tqdm(df['text'], desc="Applying Model"):
    result = predict(text)
    #print(result)
    # Append to the list
    predicted_class_hateful.append(result[0])
    predicted_class_constructive.append(result[1])
    predicted_class_agree.append(result[2])


# Add the predicted classes as a new column in the DataFrame
df['predicted_class_hateful'] = predicted_class_hateful
df['predicted_class_constructive'] = predicted_class_constructive
df['predicted_class_agree'] = predicted_class_agree


label_mappings = {
    'predicted_class_hateful': {0: 'Hateful', 1: 'Not Hateful', 2:'Unclassifiable Hateful'},
    'predicted_class_constructive': {0: 'Not Constructive', 1: 'Constructive',2: 'Unclassifiable Constructive'},
    'predicted_class_agree': {0: 'Agree', 1: 'Disagree',2:  'Unclear'}}
for column, mapping in label_mappings.items():
    df[column] = df[column].map(mapping)


# Display the DataFrame with predictions
df


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_928/2234694044.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicit

,text,predicted_class_hateful,predicted_class_constructive,predicted_class_agree
0,Test,Not Hateful,Not Constructive,Unclear
1,I hate you,Not Hateful,Not Constructive,Disagree
2,I love you,Not Hateful,Not Constructive,Agree
3,@Acosta You are more than right Jim. Such a s...,Hateful,Not Constructive,Agree
4,"i very much agree, i think that is a great point",Not Hateful,Not Constructive,Agree
5,"i think you should watch your tone, it can be ...",Not Hateful,Not Constructive,Disagree
6,Making fun of a serious health condition is no...,Not Hateful,Constructive,Disagree


In [15]:
# use if model apllied is using the custom class roberta model

import torch.nn as nn
import pandas as pd
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSequenceClassification, XLMRobertaModel
from transformers import XLMRobertaForSequenceClassification


# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-large")


# Used if the model to be applied is using the customized model 
class RobertaForContextualClassification(nn.Module):
    def __init__(self, num_labels):
        super(RobertaForContextualClassification, self).__init__() 
        self.roberta = XLMRobertaModel.from_pretrained('xlm-roberta-large') # Initializes the XLM-RoBERTa model from Hugging Face's transformers library
        self.dropout = nn.Dropout(0.1) # Adds a dropout layer (regularization technique) to to prevent overfitting = 10% of the neurons are randomly deactivated during training
        self.linear = nn.Linear(self.roberta.config.hidden_size, num_labels) # Adds a linear layer that maps hidden size (the output size of the model's hidden representations) to the number of classification labels (num_labels). This linear layer is responsible for producing the logits (predictions) for each class.

    def forward(self, input_ids, attention_mask): 
        # Processes the input IDs and attention mask through the model and returns an output
        output = self.roberta(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    inputs_embeds=None) 

        pooled_output = output[1] # Takes second element of output, the pooled output which is the hidden state of the [CLS] token
        pooled_output = self.dropout(pooled_output) # A dropout layer is applied to reduce overfitting
        logits = self.linear(pooled_output) # The pooled output is then passed through the linear layer, which produces logits for each class.
        return logits



# Load the pre-trained model architecture
#model = XLMRobertaForSequenceClassification.from_pretrained("xlm-roberta-large",  num_labels=9) #change if using customized model, to:
model = RobertaForContextualClassification(num_labels=9) 



# Load your custom saved weights
model.load_state_dict(torch.load("/work/IdaHeleneDencker#2808/Redoing_counterspeech_classifier/output/model_with_reference/checkpoints/model_epoch_2.pt"))
# Ensure the model is in evaluation mode
model.eval()




# Simple list of texts to classify
data = ['Test', 
        'I hate you', 
        'I love you',
        '@Acosta You are more than right Jim.  Such a shame that there are so many ignorant fools in this country.', 
        'i very much agree, i think that is a great point', 
        'i think you should watch your tone, it can be hurtful to other peple',
        'Making fun of a serious health condition is not the way to get your point across.']
# Convert to DataFrame
df = pd.DataFrame(data, columns=['text'])




def predict(text):
    # Tokenize the input text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    input_ids = inputs['input_ids']  # Extract input IDs
    attention_mask = inputs['attention_mask']  # Extract attention mask
    
    # Make sure to disable gradient calculation for inference
    with torch.no_grad():
        logits = model(input_ids, attention_mask=attention_mask) 

    # Split the logits for each task
    logits_hateful = logits[:, :3]          
    logits_constructive = logits[:, 3:6] 
    logits_agree = logits[:, 6:]
    
    preds_hateful = torch.argmax(logits_hateful, dim=1).item()
    preds_constructive = torch.argmax(logits_constructive, dim=1).item()
    preds_agree = torch.argmax(logits_agree, dim=1).item()
    
    return preds_hateful, preds_constructive, preds_agree



# List to store the predicted classes
predicted_class_hateful = []
predicted_class_constructive = []
predicted_class_agree = []



# Iterate over each text for prediction
for text in tqdm(df['text'], desc="Applying Model"):
    result = predict(text)
    # Append to the list
    predicted_class_hateful.append(result[0])
    predicted_class_constructive.append(result[1])
    predicted_class_agree.append(result[2])


# Add the predicted classes as a new column in the DataFrame
df['predicted_class_hateful'] = predicted_class_hateful
df['predicted_class_constructive'] = predicted_class_constructive
df['predicted_class_agree'] = predicted_class_agree


label_mappings = {
    'predicted_class_hateful': {0: 'Hateful', 1: 'Not Hateful', 2:'Unclassifiable Hateful'},
    'predicted_class_constructive': {0: 'Not Constructive', 1: 'Constructive',2: 'Unclassifiable Constructive'},
    'predicted_class_agree': {0: 'Agree', 1: 'Disagree',2:  'Unclear'}}
for column, mapping in label_mappings.items():
    df[column] = df[column].map(mapping)


# Display the DataFrame with predictions
df


/tmp/ipykernel_928/3529757259.py:44: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/work/IdaHeleneDencker#2808/Redoing_counterspeech_classi

,text,predicted_class_hateful,predicted_class_constructive,predicted_class_agree
0,Test,Unclassifiable Hateful,Unclassifiable Constructive,Unclear
1,I hate you,Not Hateful,Not Constructive,Unclear
2,I love you,Not Hateful,Constructive,Agree
3,@Acosta You are more than right Jim. Such a s...,Not Hateful,Not Constructive,Agree
4,"i very much agree, i think that is a great point",Not Hateful,Not Constructive,Agree
5,"i think you should watch your tone, it can be ...",Not Hateful,Constructive,Disagree
6,Making fun of a serious health condition is no...,Not Hateful,Constructive,Disagree
